In [44]:
%run util.py

In [45]:
data_folder = '/home/jzh/Waze/'

In [46]:
import json

In [47]:
[distToClusterCenter, loc_label, loc_centroid] = zload(data_folder + 'loc_quant_results.pkz')
[time_range, KL_list, etaWC_list, win_jam] = zload(data_folder + 'detec_results.pkz')
para_dict = zload(data_folder + 'para_dict.pkz')

In [48]:
loc_centroid

[[-71.085217,
  42.344827,
  -71.085217,
  42.347836,
  -71.0831385,
  42.344827,
  -71.0831385,
  42.347836],
 [-71.110251,
  42.335729,
  -71.110251,
  42.338452,
  -71.106477,
  42.335729,
  -71.106477,
  42.338452],
 [-71.111864,
  42.364303,
  -71.111864,
  42.366344,
  -71.105036,
  42.364303,
  -71.105036,
  42.366344]]

In [49]:
latitudes = []

for i in range(3):
    for j in range(4):
        latitudes.append(loc_centroid[i][2*j+1])

In [50]:
longitudes = []

for i in range(3):
    for j in range(4):
        longitudes.append(loc_centroid[i][2*j])

In [51]:
import gmplot

gmap = gmplot.GoogleMapPlotter(42.351848, -71.110730, 13)

gmap.scatter([42.351848], [-71.110730], c='g', marker=True)

gmap.circle(42.351848, -71.110730, 3000, '#ffefef', ew=4)

gmap.scatter(latitudes, longitudes, c='r', marker=True)

gmap.draw('./loc_clusters.html')

In [52]:
len(time_range), len(KL_list), len(etaWC_list), time_range[0:2]

(715, 715, 715, [0.0, 2.0])

In [53]:
win_size = para_dict['win_size']

alarmWinIdxList = []
alarmWinIdxStartTimeDict = {}
potential_ano_list = []
for i in range(len(KL_list)):
    if KL_list[i] > etaWC_list[i]:
        alarmWinIdxList.append(i)
        alarmWinIdxStartTimeDict[i] = 60 * time_range[i] + win_size
        for j in range(len(win_jam[str(i)])):
            potential_ano_list.append(win_jam[str(i)][j])

In [106]:
KL_list[368], etaWC_list[368], time_range[368], time_range[0], win_size, 60 * 736 + 600

(0.45456054596683332, 0.342963671068031, 736.0, 0.0, 600, 44760)

In [54]:
alarmWinIdxStartTimeDict, alarmWinIdxList

({368: 44760.0,
  369: 44880.0,
  370: 45000.0,
  371: 45120.0,
  372: 45240.0,
  423: 51360.0,
  424: 51480.0,
  425: 51600.0,
  426: 51720.0,
  427: 51840.0,
  638: 77160.0,
  639: 77280.0,
  640: 77400.0,
  641: 77520.0,
  642: 77640.0,
  653: 78960.0,
  654: 79080.0,
  655: 79200.0,
  656: 79320.0,
  657: 79440.0,
  683: 82560.0,
  684: 82680.0,
  685: 82800.0,
  686: 82920.0,
  687: 83040.0},
 [368,
  369,
  370,
  371,
  372,
  423,
  424,
  425,
  426,
  427,
  638,
  639,
  640,
  641,
  642,
  653,
  654,
  655,
  656,
  657,
  683,
  684,
  685,
  686,
  687])

In [55]:
with open(data_folder + 'sample_path_jam_test_data_sorted.json', 'r') as json_file:
    sample_path_jam_test_data_sorted = json.load(json_file)
    
with open(data_folder + 'sample_path_jam_test_data_sorted_trans.json', 'r') as json_file:
    sample_path_jam_test_data_sorted_trans = json.load(json_file)
    
with open(data_folder + 'sample_path_jam_ref_data_sorted.json', 'r') as json_file:
    sample_path_jam_ref_data_sorted = json.load(json_file)
    
with open(data_folder + 'sample_path_jam_ref_data_sorted_trans.json', 'r') as json_file:
    sample_path_jam_ref_data_sorted_trans = json.load(json_file)

In [56]:
lengthTestList = []
for i in range(len(sample_path_jam_ref_data_sorted)):
    lengthTestList.append(sample_path_jam_ref_data_sorted[str(i)]['length'])

In [107]:
mu_length = np.mean(lengthTestList)
sigma_length = np.std(lengthTestList)

# apply the three-sigma rule
thres_length = mu_length + 3 * sigma_length

In [108]:
thres_length

1921.1039361100393

In [109]:
# len(set(potential_ano_list)), set(potential_ano_list)

In [110]:
len(sample_path_jam_test_data_sorted_trans)

3881

In [111]:
# Refine the detection results; if a jam's length is larger than thres_length,
# then the jam is "indeed" abnormal.
ano_list = []
for key in set(potential_ano_list):
    if (sample_path_jam_test_data_sorted[key]['length'] > thres_length):
        ano_list.append(key)

In [112]:
len(ano_list), ano_list

(4, ['788', '3005', '742', '3657'])

In [113]:
anoStartTimeDict = {}
for key in ano_list:
    anoStartTimeDict[key] = sample_path_jam_test_data_sorted[key]['startTime'] / 60.0

In [114]:
anoStartTimeDict

{'3005': 1285.0039599999998,
 '3657': 1375.0031683333334,
 '742': 745.0015683333334,
 '788': 745.005035}

In [115]:
anoFirstAlarmTimeDict = {}
for key in ano_list:
    for winIdx in alarmWinIdxList:
        if key in win_jam[str(winIdx)]:
            anoFirstAlarmTimeDict[key] = alarmWinIdxStartTimeDict[winIdx] / 60.0
            break

In [121]:
sample_path_jam_test_data_sorted['739']['startTime'], sample_path_jam_test_data_sorted['792']['startTime']

(44700.0239, 44700.3521)

In [116]:
anoFirstAlarmTimeDict

{'3005': 1286.0, '3657': 1376.0, '742': 746.0, '788': 746.0}

In [117]:
alarmWinIdxStartTimeDict

{368: 44760.0,
 369: 44880.0,
 370: 45000.0,
 371: 45120.0,
 372: 45240.0,
 423: 51360.0,
 424: 51480.0,
 425: 51600.0,
 426: 51720.0,
 427: 51840.0,
 638: 77160.0,
 639: 77280.0,
 640: 77400.0,
 641: 77520.0,
 642: 77640.0,
 653: 78960.0,
 654: 79080.0,
 655: 79200.0,
 656: 79320.0,
 657: 79440.0,
 683: 82560.0,
 684: 82680.0,
 685: 82800.0,
 686: 82920.0,
 687: 83040.0}